# Data Ingestion mit Langchain und ChromaDB

In [1]:
# imports 
import pandas as pd
from langchain_openai import OpenAIEmbeddings # Embedding Modell
from langchain_openai import ChatOpenAI  # Chat Modell
from langchain_community.vectorstores import Chroma # Vektor-Datenbank
from langchain_core.documents import Document # Für das Document Objekt
from langchain_text_splitters import RecursiveCharacterTextSplitter # Zum Aufteilen langer Texte in Chunks
from dotenv import load_dotenv # Für das Laden des OpenAI APi Keys
import os  #  für "get .env"

In [2]:
# CSV laden
df = pd.read_csv("nasdaq_100_final_for_RAG.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ticker                  103 non-null    object 
 1   Company                 103 non-null    object 
 2   Long Business Summary   103 non-null    object 
 3   Sector (Yahoo)          102 non-null    object 
 4   Industry (Yahoo)        102 non-null    object 
 5   Country                 102 non-null    object 
 6   Market Cap              103 non-null    float64
 7   Current Price           102 non-null    float64
 8   52 Week High            103 non-null    float64
 9   Average Monthly Return  102 non-null    float64
 10  Previous Close          103 non-null    float64
 11  Dividend Yield          60 non-null     float64
 12  PE Ratio                95 non-null     float64
 13  Forward PE              102 non-null    float64
 14  PEG Ratio               0 non-null      fl

In [4]:
# PEG Ratio ist leer. Diese Spalte kann ich löschen!
df = df.drop(columns=["PEG Ratio"])

# Außerdem möchte ich keine "nan" Werte als Text in meinem RAG Dokument haben.
df = df.fillna("")  # <- alles, was NaN war, wird zum leeren String 

# Zahlen sollten außerdem Strings sein
df = df.astype(str)

Problem: Eine CSV ist KEIN embedding-fähiges Objekt. 
Eine Vektordatenbank erwartet einen langen erklärenden Text (Dokument) und eventuell Metadaten. Der Embedding-Encoder braucht einen einzigen, zusammenhängenden Text! 
--> Die Lösung: Spalten zusammenenfügen! Pro Zeile soll ein Text-Dokument erstellt werden. 

Problem 2: Zahlen 
Embeddings sind für semantischen Text optimiert. Ein RAG-System wandelt Text in Vektoren/Embeddings um. 
Reine Zahlen tragen wenig bis gar keine semantische Information. Und werden daher oft einfach nicht als relevant betrachtet. Im schlimmsten Fall „verrauschen“ die Embeddings durch viele Zahlenwerte.

-->Lösung: Zahlen in den Metadaten speichern! 
Die Metadaten (z.B. Kennzahlen wie Market Cap, Current Price, PE Ratio, ROE,und die Website) werden aber nicht automatisch vom LLM erkannt, wenn das LLM nur den Text (page_content) bekommt.

Die Zahlen aus den Metadaten können aber dynamisch in den Prompt eingefügt werden.

Das LLM bekommt dann Text + Zahlen, ohne dass die Embeddings „verschmutzt“ werden.

In [5]:
# Strukturierung der Daten für RAG

#  Zeilen werden in ein Dokument umgewandelt
def row_to_document(row):

# row["Spaltenname"] gibt den Wert der Spalte für diese Zeile zurück.
# Alles, was zwischen den Triple-Quotes (f"""    """") steht, wird  übernommen. 
# Das Ergebnis ist dann ein formatierter Text, der alle wichtigen Informationen aus dieser Zeile enthält.
    text = f"""
    Company: {row["Company"]}  
    Ticker: {row["Ticker"]}

    Business Summary: {row["Long Business Summary"]}

    Sector: {row["Sector (Yahoo)"]}
    Industry: {row["Industry (Yahoo)"]}
    Country: {row["Country"]}

    Latest News:
    Title: {row["Latest_News_Title"]}
    Summary: {row["News Summary"]}
    Sentiment: {row["Sentiment"]}
"""

    metadata = {  # Diese Metadaten sind reine Zahlen oder Links, die das LLM nicht automatisch aus dem Text extrahieren würde.
        "ticker": row["Ticker"],
        "company": row["Company"],
        "sector": row["Sector (Yahoo)"],
        "industry": row["Industry (Yahoo)"],
        "country": row["Country"],
        "market_cap": row["Market Cap"],
        "current_price": row["Current Price"],
        "previous_close": row["Previous Close"],
        "52_week_high": row["52 Week High"],
        "avg_monthly_return": row["Average Monthly Return"],
        "dividend_yield": row["Dividend Yield"],
        "pe_ratio": row["PE Ratio"],
        "forward_pe": row["Forward PE"],
        "price_to_book": row["Price to Book"],
        "total_revenue": row["Total Revenue"],
        "debt_to_equity": row["Debt to Equity"],
        "roe": row["ROE"],
        "return_1y": row["1y Return"],
        "volatility": row["Volatility"],
        "sentiment": row["Sentiment"],
        "confidence": row["Confidence"],
        "news_link": row["Latest_News_Link"],
        "latest_news_title":row["Latest_News_Title"],
        "news_summary": row["News Summary"],
        "website": row["Website"], 
    }

    return Document(page_content=text, metadata=metadata)  # Document ist ein Objekt aus LangChain.Es speichert den Text (page content) und die Metadaten. 
# Der zusammenhängende Text (page_content) wird später der in Embeddings umgewandelt. Metadaten werden später im dynamischen Prompting verwendet.

# Alle Dokumente erstellen
docs = [row_to_document(row) for i, row in df.iterrows()] # iteriert über alle Zeilen im DataFrame df. Für jede Zeile wird die Funktion row_to_document aufgerufen, um ein Document-Objekt zu erstellen. Das Ergebnis ist eine Liste von Document-Objekten, die in docs gespeichert wird.


print(docs[0]) # Zeigt das erste Document in der Liste docs an.

page_content='
    Company: Apple Inc.  
    Ticker: AAPL

    Business Summary: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple Vision Pro, Apple TV, Apple Watch, Beats products, and HomePod, as well as Apple branded and third-party accessories. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a

In [6]:
# Splitter konfigurieren
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50) # Max. Zeichen pro Chunk, Überlappung zwischen Chunks, um Kontextverlust zu vermeiden    

# leere Liste für die gesplitteten Dokumente
chunked_docs = [] 

for doc in docs: # Ich benutze eine For Schleife, weil ich eine Liste mit Dokumenten habe.
    # Text in Chunks splitten
    chunks = text_splitter.split_text(doc.page_content) # doc.page_content, weil ich die metadaten beibehalten möchte. 
    # --> Das Ergebnis ist eine Liste von Strings (Chunks)
    
    # Für jeden Chunk ein neues Document erstellen, Metadaten übernehmen.  So hat jeder Chunk des page_contents die gleichen Metadaten
    for chunk in chunks:
        chunked_docs.append(Document(page_content=chunk, metadata=doc.metadata)) # Jedes Document wird in die Liste chunked_docs hinzugefügt.


In [7]:
# Ich möchte text-embedding-3-small von OpenAI verwenden. Dafür brauche ich meinen API Key, den ich in einer .env habe. 
from dotenv import load_dotenv
import os

# .env Datei laden
load_dotenv()

# Key holen
api_key = os.getenv("OPENAI_API_KEY")

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # Auswahl des Embedding Modells

# Chroma-Datenbank erstellen
db = Chroma.from_documents(documents=chunked_docs, embedding=embeddings, collection_name="nasdaq_docs") 
#  Chroma Vektor-Datenbank aus den gesplitteten Dokumenten (chunked_docs) und den Embeddings. Der Name der Sammlung ist "nasdaq_docs".
# Die Metadaten werden zusammen mit jedem Chunk in der Chroma-Datenbank gespeichert. (passiert automatisch)

# Retrieval, Generate und Testen

In [9]:
# Imports
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

from typing import Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [10]:
# LLM Definieren
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # Temperature 0 sorgt dafür, dass mein LLM nicht halluziniert. (Nur hohe Wahrscheinlichkeiten werden als Antworten ausgegeben)

In [11]:
retriever = db.as_retriever(search_kwargs={"k": 4}) # Kritisch! K = 1 ist perfekt, wenn ich eine Aktie Analysieren möchte. 
#Wenn ich aber Aktien vergleichen will, dann wäre ein anderes K besser.  

# Ich habe an dieser stelle auch viel rumprobiert. z.B. mit Reranking oder Agentic (Vorgelagerter Knoten, der entscheided ob er ein Retriever mit k =1 oder einen mit k = 6 benutzt) 
# Das ergebnis war aber nicht besser als einfach K auf 5 zu setzen..

In [12]:
# RAG Graph erstellen      
class RAGState(TypedDict):  # Definiert die Datenstruktur  für den Graphen.
    input: str              # Feld für die ursprüngliche Benutzerfrage (ein String).
    chat_history: List[BaseMessage] # Feld für den Chat-Verlauf (eine Liste von BaseMessage-Objekten).
    context: List[Document] # Feld für die abgerufenen Daten (eine Liste von Dokument-Objekten).
    answer: str             # Feld für die finale Antwort des LLM (ein String).

In [13]:
def reformulate_query(state):
    if not state.get("chat_history"): 
        return state # Abbruch, falls kein Verlauf existiert
    
    # Prompt definieren: System-Befehl, Verlauf-Platzhalter, User-Frage
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Formuliere die Frage so um, dass sie ohne Verlauf verständlich ist (Namen statt Pronomen). Nur die Frage."), # Anweisung
        MessagesPlaceholder("history"), # Fügt Chat-Verlauf ein
        ("human", "{input}") # Aktuelle User-Frage
    ])

    # Chain (Prompt -> LLM -> Text) erstellen und direkt ausführen
    new_query = (prompt | llm | StrOutputParser()).invoke({"history": state["chat_history"], "input": state["input"]}) # Generierung
    
    state["input"] = new_query # Überschreibt die vage Frage mit der präzisen
    return state # Gibt aktualisierten State zurück

In [14]:
# Retrieve 
def retrieve(state): # Definiert den 'retrieve'-Knoten für den LangGraph. --> erhält den aktuellen "Zustand" (state). 
    docs = retriever.invoke(state["input"]) # Ruft das die Vektor-Datenbank) auf. --> und nutzt die Nutzeranfrage (state["input"]), um die relevantesten Dokumente zu finden.
    state["context"] = docs # Speichert die *kompletten* Dokumente (Text + Metadaten) und nicht nur den Text (doc.page_content for doc in docs).
    return state # Gibt den aktualisierten Zustand zurück.

In [15]:
# definiert den 'generate'-Knoten für den LangGraph. Die Funktion erhält ein state-Objekt als Eingabe
def generate(state):

    docs = state["context"]  # abgerufene Dokumente aus dem Zustand holen
    history = state.get("chat_history", []) #

    # Liste für die Kontext-Blöcke, die an das LLM übergeben werden
    prompt_blocks = []

    for doc in docs:  # Für jedes Dokument einen eigenen Block bauen
        block = "" # leerer String 

        # Textinhalt des Dokuments 
        block += doc.page_content + "\n\n"

        # Alle  relevanten Metadaten (Kennzahlen) strukturiert hinzufügen
        block += "Finanzkennzahlen & Metadaten\n"
        block += f"Ticker: {doc.metadata.get('ticker', '')}, Unternehmen: {doc.metadata.get('company', '')}\n"
        block += f"Sektor: {doc.metadata.get('sector', '')}, Industrie: {doc.metadata.get('industry', '')}\n"
        block += f"Marktkapitalisierung: {doc.metadata.get('market_cap', '')}\n"
        block += f"Aktueller Kurs: {doc.metadata.get('current_price', '')}, Vortagesschluss: {doc.metadata.get('previous_close', '')}, 52-Wochen-Hoch: {doc.metadata.get('52_week_high', '')}\n"
        block += f"KGV (PE Ratio): {doc.metadata.get('pe_ratio', '')}, KGV (Forward PE): {doc.metadata.get('forward_pe', '')}\n"
        block += f"Dividendenrendite: {doc.metadata.get('dividend_yield', '')}, Kurs-Buchwert-Verhältnis (PB): {doc.metadata.get('price_to_book', '')}\n"
        block += f"Gesamtumsatz: {doc.metadata.get('total_revenue', '')}, Verschuldungsgrad (Debt/Equity): {doc.metadata.get('debt_to_equity', '')}\n"
        block += f"Eigenkapitalrendite (ROE): {doc.metadata.get('roe', '')}, 1-Jahres-Rendite: {doc.metadata.get('return_1y', '')}, durchschnittliche monatl. Rendite der letzten 5 jahre: {doc.metadata.get('avg_monthly_return', '')}\n"
        block += f"Volatilität: {doc.metadata.get('volatility', '')}, Link zur Webseite: {doc.metadata.get('website', '')}\n\n"
        block += f"Nachrichten Titel: {doc.metadata.get('latest_news_title', '')}, Bewertung der News: {doc.metadata.get('sentiment', '')}, Link zur News: {doc.metadata.get('news_link', '')}\n"
        block += f"Inhalt: {doc.metadata.get('news_summary', '')}\n"

        prompt_blocks.append(block)  # kompletten Block in die Liste packen

    # finalen Promptkontext bauen
    context = "\n\n".join(prompt_blocks) # Liste von Strings (Text + Metadaten) = prompt_blocks, join verbindet alle Strings in der Liste zu einem einzigen String, 

    # Wie in der Demo (Prompt Engineering) # Das LLM checkt sonst z.B. nicht, dass ich die Zahlen in Euro umgeandelt habe!
    messages = [{"role": "system","content": ( "You are a helpful financial assistant who answers questions based on the given context. Analyze all financial data and metadata carefully. Bei den Kennzahlen handelt es sich um EURO")
        },*history,{"role": "user", "content": f"Context:\n{context}\n\nQuestion: {state['input']}"}]

    # LLM Aufruf
    answer = llm.invoke(messages)

    new_history = history + [
        HumanMessage(content=state["input"]),
        AIMessage(content=answer.content)
    ]

    return {
        "answer": answer.content, 
        "chat_history": new_history 
    }


In [16]:
# RAG Graph zusammenbauen 
rag = (
    StateGraph(RAGState)
    .add_node("reformulate", reformulate_query) # Chateingabe Umschreiben mit Chathistory, wenn vorhanden
    .add_node("retrieve", retrieve) # Dann Retrieve
    .add_node("generate", generate) # Dann Generate
    
    # Ablauf: Erst Umschreiben -> Dann Suchen -> Dann Antworten
    .set_entry_point("reformulate") # Startknoten ist reformulate 
    .add_edge("reformulate", "retrieve") # verbindet reformulate mit retrieve
    .add_edge("retrieve", "generate") # verbindet retrieve mit generate 
    .add_edge("generate", END) # Endknoten
    .compile() # Kompiliert den Graphen
)

Testen

In [17]:
# Globale Variable (für den Chat-Verlauf)
global_chat_history = []

def ask(question):
    """
    Die letzten 4 Nachrichten werden als Gedächtnis genutzt. 
    """
    global global_chat_history
    
    # 1. Graph aufrufen
    result = rag.invoke({
        "input": question, 
        "chat_history": global_chat_history # Übergibt den aktuellen Verlauf an den Graphen, welcher dann mit reformulate startet! Reformulate bekommt also die Chatnachrichten und die aktuelle Frage. Und erst dann wird im Rag gesucht!
    })
    
    # 2. Gedächtnis aktualisieren. Nur die letzten 4 Nachrichten werden genutzt. So wirde der Input nicht viel viel zu lang.
    global_chat_history = result["chat_history"][-4:]
    
    # 3. Antwort ausgeben
    print(f"{result['answer']}\n")
   

In [18]:
ask("hi")

Hello! How can I assist you today with financial information or analysis?



In [19]:
# testen
ask("Analysiere die Aktie Nvidea")

Die Aktie von Nvidia (Ticker: NVDA) gehört zum Technologiesektor und zur Halbleiterindustrie. Hier sind einige wichtige Finanzkennzahlen und Metadaten für Nvidia:

1. Marktkapitalisierung: 3.766.828.814.617,80 EURO
2. Aktueller Kurs: 154,98 EURO
3. Vortagesschlusskurs: 156,51 EURO
4. 52-Wochen-Hoch: 212,19 EURO
5. Kurs-Gewinn-Verhältnis (KGV): 44,28
6. KGV (Forward PE): 43,42
7. Dividendenrendite: 0,02
8. Kurs-Buchwert-Verhältnis (PB): 36,57
9. Gesamtumsatz: 162.139.825.805,72 EURO
10. Verschuldungsgrad (Debt/Equity): 8,82
11. Eigenkapitalrendite (ROE): 1,07
12. 1-Jahres-Rendite: 33,24%
13. Durchschnittliche monatliche Rendite der letzten 5 Jahre: 5,59%
14. Volatilität: 14,76%
15. Webseite: [Nvidia Webseite](https://www.nvidia.com)

Zusätzlich gibt es eine neutrale Nachricht mit dem Titel "Crucial Black Friday, AI bubble worries and Fed rate cut hopes", die auf Yahoo Finance veröffentlicht wurde. Die Nachricht diskutiert die Auswirkungen des Black Friday auf die Märkte sowie die Bedenk

In [20]:
# Testet das Gedächtnis.
ask("Ist die aktuelle News zu diesem Unternehmen eher positiv oder negativ zu bewerten?")


Die aktuelle News zu Nvidia ist positiv zu bewerten. Der Titel der Nachricht lautet "Top Semiconductor Stocks. Nvidia, AMD, Intel Poised for AI Growth", und die Zusammenfassung besagt, dass eine "AI Spending Frenzy Sparks Chip Rally" ausgelöst wurde und Nvidia weiterhin eine wichtige Position bei Investoren innehat. Der Sentiment-Wert für diese Nachricht ist ebenfalls als positiv angegeben.

Diese positive Einschätzung wird durch die Tatsache unterstützt, dass Nvidia im Technologiesektor und in der Halbleiterindustrie tätig ist, was aufgrund des wachsenden Interesses an künstlicher Intelligenz (AI) und der steigenden Nachfrage nach Halbleiterlösungen eine vielversprechende Position darstellt. Insgesamt deutet die News darauf hin, dass Nvidia gut positioniert ist, um vom erwarteten Wachstum im Bereich der künstlichen Intelligenz zu profitieren.



Man könnte es auch einfach ohne Graph machen und eine semantische Suche durchführen. Hier könnte man aber nicht genau steuern was passiert. Es wäre Linear und nicht modular.
Man hätte die Chat History anders einbauen müssen!

In [22]:
# RAG Test abfrage:
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# query = "Analysiere Apple, gehe auch auf aktuelle News ein. Sind diese Positiv oder negativ für die Aktie?"

# Top-k relevante Chunks abrufen
# results = db.similarity_search(query, k=1)

# Dieser Block nutzt jetzt alle relevanten Metadaten
# prompt_context = ""
# for doc in results:
    # 1. Semantischer Text-Inhalt
    #prompt_context += doc.page_content + "\n"
    
    # 2. Alle relevanten Metadaten strukturiert hinzufügen
    #prompt_context += f"--- Finanzkennzahlen & Metadaten ---\n"
    #prompt_context += f"Ticker: {doc.metadata['ticker']}, Unternehmen: {doc.metadata['company']}\n"
    #prompt_context += f"Sektor: {doc.metadata['sector']}, Industrie: {doc.metadata['industry']}\n"
    #prompt_context += f"Marktkapitalisierung: {doc.metadata['market_cap']}\n"
    #prompt_context += f"Aktueller Kurs: {doc.metadata['current_price']}, Vortagesschluss: {doc.metadata['previous_close']}\n"
    #prompt_context += f"KGV (PE Ratio): {doc.metadata['pe_ratio']}, KGV (Forward PE): {doc.metadata['forward_pe']}\n"
    #prompt_context += f"Dividendenrendite: {doc.metadata['dividend_yield']}, Kurs-Buchwert-Verhältnis (PB): {doc.metadata['price_to_book']}\n"
    #prompt_context += f"Gesamtumsatz: {doc.metadata['total_revenue']}, Verschuldungsgrad (Debt/Equity): {doc.metadata['debt_to_equity']}\n"
    #prompt_context += f"Eigenkapitalrendite (ROE): {doc.metadata['roe']}, 1-Jahres-Rendite: {doc.metadata['return_1y']}\n"
    #prompt_context += f"Volatilität: {doc.metadata['volatility']}, Link zur Webseite: {doc.metadata['website']}\n\n"
    #prompt_context += f"Nachrichten Titel: {doc.metadata['latest_news_title']}, Medienstimmung diesbezüglich: {doc.metadata['sentiment']}, Link zur News: {doc.metadata['news_link']}\n"

#prompt = f"""
#Nutze die folgenden Informationen, um eine Analyse zu erstellen. 
#Berücksichtige sowohl den beschreibenden Text als auch die detaillierten Kennzahlen:

#{prompt_context}
#"""

# Korrekter Aufruf mit .invoke()
#response = llm.invoke(prompt) 

#print(response.content)